In [1]:
import numpy as np
import xarray as xr
#import multiprocessing
from xarray.backends import NetCDF4DataStore
#from google.colab import drive
#drive.mount('/content/gdrive')

In [2]:
pip install netCDF4

     |████████████████████████████████| 4.7MB 14.2MB/s 
     |████████████████████████████████| 317kB 43.5MB/s 


In [3]:
import netCDF4

In [13]:
import wget

In [12]:
pip install wget

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=d26d132cf0bea213e34f01da3fcd56cde445078b51bfb3c7068b42212f2d49c5
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [19]:
wget_url = "https://portal.nccs.nasa.gov/datashare/NEXGDDP/BCSD/rcp85/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_rcp85_r1i1p1_ACCESS1-0_2006.nc"

In [22]:
f = wget.download(wget_url)

In [30]:
a = xr.open_dataarray(f)

In [76]:
a.shape

(365, 720, 1440)

In [73]:
dangerous_heat = (a > 318.15) * 1

In [66]:
def count_runs(arr, min_length):                  # [1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1]  original array
  if np.sum(arr) == 0:  # immediately deal with all-zero case
    return 0
  diff = np.diff(arr, prepend=0)                  # [1,  0, -1,  1,  0,  0,  0, -1,  0,  1, -1,  1]  1 if 0 precedes 1 or 1 is first element, -1 if 1 precedes 0
  starts = np.cumsum(diff == 1) * (diff == 1)     # [1, 0, 0, 2, 0, 0, 0, 0, 0, 3, 0, 4]  unique label for each run start
  ends = np.cumsum(diff == -1) * (diff == -1)     # [0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 3, 0]  correspondng label for each run end
  start_positions = (np.argwhere(starts))[:,0]    # [ 0,  3,  9, 11]  positions of each run start
  end_positions = (np.argwhere(ends))[:,0]        # [ 2,  7, 10]      positions of each run end
  if end_positions.size < start_positions.size:
    end_positions = np.append(end_positions, len(arr) + 1)        # [ 2,  7, 10, 12]  deal with case of 1 in rightmost position
  run_lengths = [end_positions[i] - start_positions[i] for i in range(len(start_positions))]  # [2, 4, 1, 1]
  return len([i for i in run_lengths if i < min_length])

In [74]:
runs = np.apply_along_axis(count_runs, 0, dangerous_heat, (5,))

In [75]:
runs.shape

(720, 1440)